In [ ]:
!pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
def prepare_dicom_image(image_path: str,method,cL=5.0,tile=(8,8),patch=5,mode="gray"):
    # Читаем .dcm
    ds=dicom.dcmread(image_path)
    # Переводим в ndarray для обработки с помощью cv2
    dcm_sample = ds.pixel_array.astype(np.float32)
    # print(ds.pixel_array.shape)

    center = getattr(ds, 'WindowCenter', None)
    if center is None:
        window_center = np.mean(ds.pixel_array)
    elif isinstance(center, dicom.multival.MultiValue):
        window_center = int(center[0]) if 'WindowCenter' in ds else np.mean(ds.pixel_array)
    else:
        window_center = int(center) if 'WindowCenter' in ds else np.mean(ds.pixel_array)

    width = getattr(ds, 'WindowWidth', None)
    if width is None:
        window_width = np.max(ds.pixel_array) - np.min(ds.pixel_array)
    elif isinstance(ds.WindowWidth, dicom.multival.MultiValue):
        window_width = int(ds.WindowWidth[0]) if 'WindowWidth' in ds else np.max(ds.pixel_array) - np.min(ds.pixel_array)
    else:
        window_width = int(ds.WindowWidth) if 'WindowWidth' in ds else np.max(ds.pixel_array) - np.min(ds.pixel_array)

    dcm_sample = apply_window_level(dcm_sample, window_center, window_width, ds.PhotometricInterpretation,method,cL,tile,patch,mode)
    return dcm_sample

In [ ]:
def prepare_jpg_data(image_path: str,method,cL=5.0,tile=(8,8),patch=5,mode="gray", output_path):
    ds=dicom.dcmread(image_path)
    dcm_sample = ds.pixel_array.astype(np.float32)
    #dcm_sample = prepare_dicom_image(image_path: str,method,cL=5.0,tile=(8,8),patch=5,mode="gray"):
    img = Image.fromarray(pixel_array)
    img.save(output_path)


In [ ]:
import glob
import pydicom
from PIL import Image
import numpy as np

def batch_dicom_to_jpg(input_folder, output_folder, quality=90):
    """
    Пакетная конвертация всех DICOM файлов в папке в JPG
    """
    # Создание папки для результатов
    os.makedirs(output_folder, exist_ok=True)
    
    # Поиск всех DICOM файлов
    dicom_files = glob.glob(os.path.join(input_folder, "*.dcm"))
    
    for dicom_file in dicom_files:
        try:
            # Чтение DICOM
            ds = pydicom.dcmread(dicom_file)
            pixel_array = ds.pixel_array
            
            # Нормализация
            if pixel_array.dtype != np.uint8:
                pixel_array = ((pixel_array - pixel_array.min()) / 
                              (pixel_array.max() - pixel_array.min()) * 255).astype(np.uint8)
            
            # Создание имени файла
            base_name = os.path.splitext(os.path.basename(dicom_file))[0]
            jpg_path = os.path.join(output_folder, f"{base_name}.jpg")
            
            # Сохранение как JPG
            img = Image.fromarray(pixel_array)
            if img.mode != 'RGB':
                img = img.convert('RGB')
                
            img.save(jpg_path, 'JPEG', quality=quality)
            print(f"✅ Конвертирован: {base_name}.dcm → {base_name}.jpg")
            
        except Exception as e:
            print(f"❌ Ошибка с {dicom_file}: {e}")

# Использование
batch_dicom_to_jpg("dicom_folder/", "jpg_output/", quality=85)